# Examples of transformations from raw AC values to other formats

These examples start with raw data as it might be provided by Audubon Core and transform it to other formats like W3C Media Fragments, W3C Web Annotations, and IIIF manifests

In [1]:
# Configuration and function definitions

import pandas as pd
import json

def build_spatial_media_fragment(x_frac, y_frac, w_frac, h_frac, x_dim, y_dim):
    # calculate absolute pixel values from percents
    x_pix = int(round(x_frac*x_dim))
    y_pix = int(round(y_frac*y_dim))
    w_pix = int(round(w_frac*x_dim))
    h_pix = int(round(h_frac*y_dim))
    fragment = 'xywh=pixel:' + str(x_pix) + ',' + str(y_pix) + ',' + str(w_pix) + ',' + str(h_pix)
    return fragment
    
def build_iiif_image_server_url(x_frac, y_frac, w_frac, h_frac, x_dim, y_dim, base_url):
    # calculate absolute pixel values from percents
    x_pix = int(round(x_frac*x_dim))
    y_pix = int(round(y_frac*y_dim))
    w_pix = int(round(w_frac*x_dim))
    h_pix = int(round(h_frac*y_dim))
    url = base_url + str(x_pix) + ',' + str(y_pix) + ',' + str(w_pix) + ',' + str(h_pix) + '/full/0/default.jpg'
    return url
    
def build_spatial_annotation(media_fragment, image_source, purpose, target, annotation_id):
    annotation_string = '''{ 
"@context": "http://www.w3.org/ns/anno.jsonld",
 "id": "''' + annotation_id + '''",
 "type": "Annotation", 
 "body": {
    "source": "''' + image_source + '''", 
    "purpose": "''' + purpose + '''", 
    "selector": { 
        "type": "FragmentSelector", 
        "conformsTo": "http://www.w3.org/TR/media-frags/", 
        "value": "'''+ media_fragment + '''" 
        } 
    },
 "target": "''' + target + '''" 
}
'''
    return annotation_string

def build_iiif_canvas__description_annotation(annotation_id, body_html, canvas_base_iri, media_fragment):
    annotation_string = '''{ 
  "id": "''' + annotation_id + '''",
  "type": "Annotation",
  "motivation": "describing",
  "body": {
    "type": "TextualBody",
    "value": "''' + body_html + '''",
    "format": "text/html"
  },
  "target": {
    "id": "''' + canvas_base_iri + '#' + media_fragment + '''",
    "type": "Canvas"
  }
}'''
    return annotation_string

# Hacked from annotated IIIF manifest example at https://stephenwf.github.io/ocean-liners.json
def build_iiif_manifest(x_dim, y_dim, media_type, image_url, canvas_base_iri, base_annotation_id, annotations):
    manifest = '''{
  "@context": [
    "http://iiif.io/api/presentation/3/context.json",
    "http://www.w3.org/ns/anno.jsonld"
  ],
  "id": "https://iiif.vam.ac.uk/collections/O1023003/manifest.json",
  "type": "Manifest",
  "viewingDirection": "left-to-right",
  "behavior": [
    "individuals"
  ],
  "label": {
    "en": [
      "Cunard Line - to all parts of the world"
    ]
  },
  "items": [
    {
      "items": [
        {
          "items": [
            {
              "body": {
                "service": [
                  {
                    "profile": "level1",
                    "type": "ImageService2",
                    "id": "https://framemark.vam.ac.uk/collections/2013GU2911"
                  }
                ],
                "format": "''' + media_type + '''",
                "height": ''' + str(y_dim) + ''',
                "width": ''' + str(x_dim) + ''',
                "type": "Image",
                "id": "''' + image_url + '''"
              },
              "motivation": "painting",
              "type": "Annotation",
              "target": "''' + canvas_base_iri + '''"
            }
          ],
          "type": "AnnotationPage"
        }
      ],
      "label": {
        "en": [
          "Object image 0"
        ]
      },
      "width": ''' + str(x_dim) + ''',
      "height": ''' + str(y_dim) + ''',
      "type": "Canvas",
      "id": "''' + canvas_base_iri + '''",
      "annotations": [
        {
          "id": "''' + base_annotation_id + '''",
          "type": "AnnotationPage",
          "items": [
''' + annotations + '''
          ]
        }
      ]
    }
  ]
}
'''
    return manifest


# Load data about abstract image and service access points

These are hand-built files using a potential JSON-LD serialization for expressing SAPs as child objects to the abstract image.

Note: the basic image metadata includes the properties required for an "Audubon Core record": identifier, type, metadataLanguage, rights.

In [4]:
# Live plant image of Tragia cordata
filepath = 'via_project_13Apr2021_12h53m_csv.csv'
base_url = 'https://zenodo.org/api/iiif/v2/2180af36-6fea-424a-b6b4-b59f3c862c51:b535ccd4-2869-4e84-9e19-1df68c7abcb4:trco--fr040529-17e5384.jpg/'
ac_jsonld = 'image_graph_hess.json'
image_id = 'http://bioimages.vanderbilt.edu/hessd/e5384'

# Fig 1 from Multiple parasitic crustacean infestation on belonid fish Strongylura strongylura
#filepath = 'via_project_13Apr2021_21h56m_csv.csv'
#base_url = 'https://zenodo.org/api/iiif/v2/87a31994-dadd-4325-b7a5-d69998c468d3:264ea952-bbf0-4f51-886d-ab95fef1292c:big_34616.jpg/'
#ac_jsonld = 'image_graph_zookeys.json'
#image_id = 'https://doi.org/10.3897/zookeys.457.6817.figure1'

with open(ac_jsonld, 'rt', encoding='utf-8') as file_object:
    text = file_object.read()
record = json.loads(text)
variants = record['ac:hasServiceAccessPoint']

# This is set to the Best Quality service access point
variant_index = 0
variant = variants[variant_index]

print(json.dumps(record, indent = 2))

{
  "@context": {
    "ac": "http://rs.tdwg.org/ac/terms/",
    "dc": "http://purl.org/dc/elements/1.1/",
    "dcterms": "http://purl.org/dc/terms/",
    "exif": "http://ns.adobe.com/exif/1.0/"
  },
  "@id": "http://bioimages.vanderbilt.edu/hessd/e5384",
  "@type": "http://purl.org/dc/dcmitype/StillImage",
  "dcterms:title": "Tragia cordata (Euphorbiaceae) - fruit \u2013 juvenile",
  "dcterms:identifier": "http://bioimages.vanderbilt.edu/hessd/e5384",
  "dcterms:type": "http://purl.org/dc/dcmitype/StillImage",
  "ac:metadataLanguage": "ttp://id.loc.gov/vocabulary/iso639-2/eng",
  "dc:rights": "(c) 2008 Darel Hess",
  "ac:hasServiceAccessPoint": [
    {
      "ac:accessURI": "https://zenodo.org/record/4683346/files/trco--fr040529-17e5384.jpg",
      "dc:format": "image/jpeg",
      "ac:variant": "http://rs.tdwg.org/acvariant/values/v006",
      "ac:variantLiteral": "Best Quality",
      "exif:PixelYDimension": 2112,
      "exif:PixelXDimension": 2526
    },
    {
      "ac:accessURI": "

# Load bounding box data created by VIA browser tool


In [5]:
via_data = pd.read_csv(filepath, na_filter=False)
x_dim = variant['exif:PixelXDimension']
y_dim = variant['exif:PixelYDimension']
decimals = len(str(x_dim)) + 1

data = []
for index, row in via_data.iterrows():
    row_dict = {'image_identifier': image_id}
    shape = json.loads(row['region_shape_attributes'])
    attributes = json.loads(row['region_attributes'])
    row_dict['feature'] = attributes['name']
    row_dict['xFrac'] = round(shape['x']/x_dim, decimals)
    row_dict['yFrac'] = round(shape['y']/y_dim, decimals)
    row_dict['widthFrac'] = round(shape['width']/x_dim, decimals)
    row_dict['heightFrac'] = round(shape['height']/y_dim, decimals)
    row_dict['base_url'] = base_url
    data.append(row_dict)
    
subdivisions = pd.DataFrame(data)
subdivisions

,image_identifier,feature,xFrac,yFrac,widthFrac,heightFrac,base_url
0,http://bioimages.vanderbilt.edu/hessd/e5384,mine,0.28939,0.23674,0.09066,0.26373,https://zenodo.org/api/iiif/v2/2180af36-6fea-4...
1,http://bioimages.vanderbilt.edu/hessd/e5384,fruit,0.21892,0.44792,0.28147,0.34612,https://zenodo.org/api/iiif/v2/2180af36-6fea-4...


# Display various formats

## Generate media fragments

In [6]:
for index, irow in subdivisions.iterrows():
    frag = build_spatial_media_fragment(irow['xFrac'], 
                         irow['yFrac'],
                         irow['widthFrac'], 
                         irow['heightFrac'], 
                         variant['exif:PixelXDimension'], 
                         variant['exif:PixelYDimension'])

    print('fragment:', irow['feature'])
    print('variant:', variant['ac:variantLiteral'])
    print('media fragment:', frag)
    print()

fragment: mine
variant: Best Quality
media fragment: xywh=pixel:731,500,229,557

fragment: fruit
variant: Best Quality
media fragment: xywh=pixel:553,946,711,731



## Generate URLs in IIIF image server format

In [7]:
for index, irow in subdivisions.iterrows():
    url = build_iiif_image_server_url(irow['xFrac'], 
                         irow['yFrac'],
                         irow['widthFrac'], 
                         irow['heightFrac'], 
                         variant['exif:PixelXDimension'], 
                         variant['exif:PixelYDimension'], 
                         irow['base_url'])

    print('fragment:', irow['feature'])
    #print('variant:', variant['ac:variantLiteral'])
    print(url)
    print()

fragment: mine
https://zenodo.org/api/iiif/v2/2180af36-6fea-424a-b6b4-b59f3c862c51:b535ccd4-2869-4e84-9e19-1df68c7abcb4:trco--fr040529-17e5384.jpg/731,500,229,557/full/0/default.jpg

fragment: fruit
https://zenodo.org/api/iiif/v2/2180af36-6fea-424a-b6b4-b59f3c862c51:b535ccd4-2869-4e84-9e19-1df68c7abcb4:trco--fr040529-17e5384.jpg/553,946,711,731/full/0/default.jpg



## General W3C web annotation

This isn't any real thing that anyone actually would use, but is just an example

In [8]:
target = 'https://www.gbif.org/occurrence/930742101'
annotation_id = 'http://example.org/darel_hess'
purpose = 'http://rs.tdwg.org/annotations/values/documenting'
anno = build_spatial_annotation(frag, variant['ac:accessURI'], purpose, target, annotation_id)
print(anno)

{ 
"@context": "http://www.w3.org/ns/anno.jsonld",
 "id": "http://example.org/darel_hess",
 "type": "Annotation", 
 "body": {
    "source": "https://zenodo.org/record/4683346/files/trco--fr040529-17e5384.jpg", 
    "purpose": "http://rs.tdwg.org/annotations/values/documenting", 
    "selector": { 
        "type": "FragmentSelector", 
        "conformsTo": "http://www.w3.org/TR/media-frags/", 
        "value": "xywh=pixel:553,946,711,731" 
        } 
    },
 "target": "https://www.gbif.org/occurrence/930742101" 
}



## Generate a IIIF manifest using annotations to describe fragments

This manifest is hack of https://stephenwf.github.io/ocean-liners.json just to show how W3C Web Annotations describing the fragments can be insterted into a IIIF manifest to highlight the fragments (if the IIIF viewer supports that). I don't think it actually works, because I think it has errors.

Note: the original example has an error in the context: http://www.w3.org/ns.anno.jsonld should be http://www.w3.org/ns/anno.jsonld that was preventing it from validating as JSON-LD. The error has been fixed in this example.

In [9]:
base_annotation_id = 'https://iiif.vam.ac.uk/collections/O1023003/annopage/p1'

annotations = ''
# Build an annotation for each subdivision
for index, row in subdivisions.iterrows():
    annotation_id = base_annotation_id + '/a' + str(index)
    body_html = '''<h2>'''+ row['feature'] +'''</h2><div>This is part of a larger image.</div>'''
    canvas_base_iri = 'https://iiif.vam.ac.uk/collections/O1023003/canvas/c0'

    frag = build_spatial_media_fragment(row['xFrac'], 
                     row['yFrac'],
                     row['widthFrac'],
                     row['heightFrac'], 
                     variant['exif:PixelXDimension'],
                     variant['exif:PixelYDimension'])
    anno = build_iiif_canvas__description_annotation(annotation_id, body_html, canvas_base_iri, frag)
    annotations += anno + ',\n'

# Strip off final trailing comma and newline
annotations = annotations[:len(annotations)-2]

manifest = build_iiif_manifest(variant['exif:PixelXDimension'], variant['exif:PixelYDimension'], variant['dc:format'], variant['ac:accessURI'], canvas_base_iri, base_annotation_id, annotations)
print(manifest)

{
  "@context": [
    "http://iiif.io/api/presentation/3/context.json",
    "http://www.w3.org/ns/anno.jsonld"
  ],
  "id": "https://iiif.vam.ac.uk/collections/O1023003/manifest.json",
  "type": "Manifest",
  "viewingDirection": "left-to-right",
  "behavior": [
    "individuals"
  ],
  "label": {
    "en": [
      "Cunard Line - to all parts of the world"
    ]
  },
  "items": [
    {
      "items": [
        {
          "items": [
            {
              "body": {
                "service": [
                  {
                    "profile": "level1",
                    "type": "ImageService2",
                    "id": "https://framemark.vam.ac.uk/collections/2013GU2911"
                  }
                ],
                "format": "image/jpeg",
                "height": 2112,
                "width": 2526,
                "type": "Image",
                "id": "https://zenodo.org/record/4683346/files/trco--fr040529-17e5384.jpg"
              },
              "motivation": "